### Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torch.utils.data as data
import matplotlib.pyplot as plt
import IPython.display
from IPython.display import Audio
import torch.optim as optim
from types import SimpleNamespace
import scipy.signal as sc
import time
from sklearn.metrics import f1_score
import pickle

from trainDataset import TrainDataset
from testDataset import TestDataset
#from trainDatasetNew import TrainDatasetNew
#from testDatasetNew import TestDatasetNew
from validation_split import get_dataloaders
from math_utils import logMagStft, ffts
from SpectrogramCNN import SpectrogramCNN
from train_utils import train, test, save_output, save_geometric_mean_predictions
from evaluation_utils import get_mean_F1
from MulitScale1DResNet import MSResNet
from LSTM import LSTM
from FFTMulitScale1DResNet import FFTMSResNet
from SpectralResNet import SpectralResNet34
from scipy.signal import hilbert
from FeatureFNN import FeatureFNN
from MelRawCombined import CNN1D, CNN2D

### Parameters

In [2]:
validation_split = .2
do_plots = False
load_model = False
args = SimpleNamespace(batch_size=32, test_batch_size=32, epochs=7,
                       lr=0.01, momentum=0.5, seed=1, log_interval=200, 
                      net = CNN1D) #SpectrogramCNN, MSResNet, SpectralResNet34, FFTMSResNet, FeatureFNN, CNN1D, CNN2D 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not torch.cuda.is_available(): # adapt those paths on other machine
    print('no cuda')
    path_train = './../data/train-small/'
    path_test =  './../data/test/kaggle-test/'
else:
    print('with cuda')
    path_train = './../data/kaggle-train/'
    path_test =  './../data/kaggle-test/'
    
path_model = 'models/model.pt'
path_submission = 'submissions/'
    
sample_rate = 16000
nmbr_classes = 10

with cuda


### Original Dataset

In [3]:
# todo add in the classes the features and the fft data

toFloat = transforms.Lambda(lambda x: x / np.iinfo(np.int16).max)

trainDataset = TrainDataset(path_train, transform=toFloat)
print(len(trainDataset))

testDataset = TestDataset(path_test, transform=toFloat)
print(len(testDataset))

269776
4096


In [4]:
input_size = len(trainDataset[0][0])
print('input size: ',input_size)

input size:  64000


### Look at Original Data

In [5]:
if do_plots:
    # how many instruments are there?
    dummy_count = np.zeros(20)

    for sample in trainDataset:
        dummy_count[sample[1]] += 1

    labels_count = []
    for elem in dummy_count:
        if elem != 0:
            labels_count.append(elem)

    print(labels_count)

In [6]:
if do_plots:
    nmbr_classes = len(labels_count)
    print('nmbr_classes: ', nmbr_classes)

In [7]:
if do_plots:
    plt.plot(labels_count, '*')

In [8]:
if do_plots:
    # plot one of each

    done = np.zeros(nmbr_classes)
    examples = []

    for sample in trainDataset:
        if done[sample[1]] == 0:
            examples.append(sample)
            done[sample[1]] = 1

In [9]:
plt.rcParams["figure.figsize"] = (20,10)

In [10]:
#time series

if do_plots:

    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        plt.plot(examples[i][0])

In [11]:
#envelope

if do_plots:
    
    m = nn.MaxPool1d(50)

    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        env = torch.from_numpy(np.abs(hilbert(examples[i][0]))).view(1,1,-1)
        env = m(env)
        plt.plot(env[0,0].numpy())
        
    plt.show()

In [12]:
# fft

if do_plots:

    m = nn.MaxPool1d(50)

    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        fft = torch.from_numpy(ffts(examples[i][0])).view(1,1,-1)
        fft = m(fft)
        plt.plot(fft[0,0].numpy())
        
    plt.show()

In [13]:
# melspec

if do_plots:
    n_fft = 510
    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        spectrogram = logMagStft(examples[i][0], 16000, n_fft)
        plt.imshow(spectrogram)
        
    plt.show()

In [14]:
if do_plots:

    for sample in examples:
        display(Audio(sample[0], rate=sample_rate))

### Dataloaders

In [15]:
# validation split is done here

train_loader, validation_loader, indices = get_dataloaders(trainDataset, 
                                                  batch_size = args.batch_size, 
                                                  validation_split = validation_split, 
                                                  shuffle_dataset = True, 
                                                  random_seed = None)
                                                    #indices = indices)

for samples, instrument_family_target in train_loader:
        print(samples.shape, instrument_family_target.shape,
              instrument_family_target.data)
        print(torch.min(samples), torch.max(samples))
        print(trainDataset.transformInstrumentsFamilyToString(instrument_family_target.data))
        break
        
for samples, instrument_family_target in validation_loader:
        print(samples.shape, instrument_family_target.shape,
              instrument_family_target.data)
        print(torch.min(samples), torch.max(samples))
        print(trainDataset.transformInstrumentsFamilyToString(instrument_family_target.data))
        break

torch.Size([32, 64000]) torch.Size([32]) tensor([0, 4, 3, 0, 4, 6, 0, 6, 5, 3, 3, 0, 5, 3, 8, 6, 4, 0, 9, 3, 0, 0, 0, 4,
        0, 2, 8, 1, 0, 6, 0, 5])
tensor(-0.9539, dtype=torch.float64) tensor(0.9462, dtype=torch.float64)
['bass' 'keyboard' 'guitar' 'bass' 'keyboard' 'organ' 'bass' 'organ'
 'mallet' 'guitar' 'guitar' 'bass' 'mallet' 'guitar' 'string' 'organ'
 'keyboard' 'bass' 'vocal' 'guitar' 'bass' 'bass' 'bass' 'keyboard' 'bass'
 'flute' 'string' 'brass' 'bass' 'organ' 'bass' 'mallet']
torch.Size([32, 64000]) torch.Size([32]) tensor([5, 3, 0, 6, 5, 3, 4, 1, 0, 0, 3, 1, 6, 0, 1, 0, 4, 8, 8, 9, 6, 8, 6, 5,
        5, 6, 0, 2, 3, 0, 0, 4])
tensor(-0.9484, dtype=torch.float64) tensor(0.9467, dtype=torch.float64)
['mallet' 'guitar' 'bass' 'organ' 'mallet' 'guitar' 'keyboard' 'brass'
 'bass' 'bass' 'guitar' 'brass' 'organ' 'bass' 'brass' 'bass' 'keyboard'
 'string' 'string' 'vocal' 'organ' 'string' 'organ' 'mallet' 'mallet'
 'organ' 'bass' 'flute' 'guitar' 'bass' 'bass' 'keyboard']


In [16]:
# save indices
with open(path_submission + "indices.txt", "wb") as fp:
    pickle.dump([indices], fp)

In [17]:
test_loader = data.DataLoader(testDataset, batch_size=args.batch_size, shuffle=False) #!!! shuffle should be false
for samples in test_loader:
        print(samples[0].shape)
        print(torch.min(samples[0]), torch.max(samples[0]))
        break

torch.Size([32, 64000])
tensor(-0.9471, dtype=torch.float64) tensor(0.9469, dtype=torch.float64)


### Main

In [18]:
if (args.net == SpectrogramCNN) or (args.net == SpectralResNet34) \
    or (args.net == FeatureFNN) or (args.net == CNN1D) \
    or (args.net == CNN2D):
    model = args.net(device).to(device)

In [19]:
if (args.net == MSResNet) or (args.net == FFTMSResNet):
    model = args.net(1, device).to(device)

In [20]:
if args.net == LSTM:
    model = args.net(device, input_size = 252, hidden_size = 512, num_layers = 1, num_classes = 10).to(device)

In [21]:
print(model)

CNN1D(
  (conv_1): Sequential(
    (0): Conv1d(1, 16, kernel_size=(9,), stride=(1,))
    (1): ReLU()
    (2): Conv1d(16, 16, kernel_size=(9,), stride=(1,))
    (3): ReLU()
    (4): MaxPool1d(kernel_size=16, stride=16, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.1)
  )
  (conv_2): Sequential(
    (0): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
    (1): ReLU()
    (2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
    (3): ReLU()
    (4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.1)
  )
  (conv_3): Sequential(
    (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
    (1): ReLU()
    (2): Conv1d(32, 32, kernel_size=(3,), stride=(1,))
    (3): ReLU()
    (4): MaxPool1d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (5): Dropout(p=0.1)
  )
  (conv_4): Sequential(
    (0): Conv1d(32, 256, kernel_size=(3,), stride=(1,))
    (1): ReLU()
    (2): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
    (

In [22]:
# Main
optimizer = optim.Adam(model.parameters(), lr=args.lr)

info = {'highest F1' : 0,
        'saved epoch' : None}

In [ ]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch, start_time = time.time())
    f1 = get_mean_F1(model, validation_loader)
    print('after epoch {} got f1 score of {}'.format(epoch , f1))
    if f1 > info['highest F1']:
        info['highest F1'] = np.copy(f1)
        info['saved epoch'] = epoch 
        test(args, model, device, test_loader, epoch, trainDataset, testDataset, path_submission)
        torch.save(model, path_model)
        print('currently best model --> saved')
        
print(info)

Train Epoch: 1 [0/269776 (0%)]	Loss: 2.321663	F1: 0.0000	Runtime: 0.3
Train Epoch: 1 [6400/269776 (3%)]	Loss: 1.984412	F1: 0.3438	Runtime: 9.6
Train Epoch: 1 [12800/269776 (6%)]	Loss: 2.081220	F1: 0.2812	Runtime: 18.7
Train Epoch: 1 [19200/269776 (9%)]	Loss: 1.903527	F1: 0.3125	Runtime: 27.8
Train Epoch: 1 [25600/269776 (12%)]	Loss: 2.110596	F1: 0.2188	Runtime: 36.9
Train Epoch: 1 [32000/269776 (15%)]	Loss: 1.859639	F1: 0.3438	Runtime: 45.9
Train Epoch: 1 [38400/269776 (18%)]	Loss: 1.776582	F1: 0.2812	Runtime: 55.1
Train Epoch: 1 [44800/269776 (21%)]	Loss: 1.726350	F1: 0.3438	Runtime: 64.2
Train Epoch: 1 [51200/269776 (24%)]	Loss: 1.854845	F1: 0.3125	Runtime: 73.3
Train Epoch: 1 [57600/269776 (27%)]	Loss: 1.985910	F1: 0.2188	Runtime: 82.4
Train Epoch: 1 [64000/269776 (30%)]	Loss: 1.507452	F1: 0.4062	Runtime: 91.5
Train Epoch: 1 [70400/269776 (33%)]	Loss: 1.722697	F1: 0.3438	Runtime: 100.6
Train Epoch: 1 [76800/269776 (36%)]	Loss: 1.787019	F1: 0.3125	Runtime: 109.6
Train Epoch: 1 [83200

### Load Model

In [ ]:
if load_model:
    model = torch.load(path_model)

In [ ]:
# load indices
with open(path_submission + "indices.txt", "rb") as fp:
    indices = pickle.load(fp)

In [ ]:
get_mean_F1(model, validation_loader)

In [ ]:
epoch=10
test(args, model, device, test_loader, epoch, trainDataset, testDataset, path_submission)